<a href="https://colab.research.google.com/github/hemsmalli5/Final-Project---Week1/blob/master/Project_Master1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Movie Data Analysis***

In [3]:
# Import dependencies
#from google.colab import files
import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer, StandardScaler
from sklearn.model_selection import train_test_split
#import tensorflow as tf

# The below ETL process was used to find the tcosnt filter list that is used to clean the 4 individual data files before they are brought into the SQL DB.

## Import Data Files

In [4]:
# Upload file
uploaded = files.upload()

NameError: name 'files' is not defined

In [5]:
# read the data file
movie_akas = pd.read_csv('title.akas.tsv', sep='\t')
movie_akas

FileNotFoundError: [Errno 2] File title.akas.tsv does not exist: 'title.akas.tsv'

In [ ]:
# read the data file
movie_basics = pd.read_csv('title.basics.tsv', sep='\t')
movie_basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
7221672,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010,\N,\N,"Action,Drama,Family"
7221673,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,\N,\N,"Action,Drama,Family"
7221674,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,"Action,Drama,Family"
7221675,tt9916856,short,The Wind,The Wind,0,2015,\N,27,Short


In [ ]:
# read the data file
movie_ratings = pd.read_csv('title.ratings.tsv', sep='\t')
movie_ratings

,tconst,averageRating,numVotes
0,tt0000001,5.6,1648
1,tt0000002,6.1,198
2,tt0000003,6.5,1352
3,tt0000004,6.2,120
4,tt0000005,6.2,2139
...,...,...,...
1079287,tt9916580,7.2,5
1079288,tt9916690,6.6,5
1079289,tt9916720,6.0,61
1079290,tt9916766,6.9,14


In [ ]:
# read the data file
movie_crew = pd.read_csv('title.crew.tsv', sep='\t')
movie_crew

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N
...,...,...,...
7221672,tt9916848,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
7221673,tt9916850,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
7221674,tt9916852,"nm5519454,nm5519375","nm6182221,nm1628284,nm2921377"
7221675,tt9916856,nm10538645,nm6951431


## Clean Akas File For Merging

In [ ]:
#Make copy of df for editing
movie_akas_2 = movie_akas.copy()

In [ ]:
# Drop all rows that do not have region as US
movie_akas_2 = (movie_akas_2.loc[movie_akas_2['region'] == 'US'])

# Drop all rows where types is alternative
movie_akas_2 = movie_akas_2[movie_akas_2.types != 'alternative']

# Drop duplicate rows if there is more then one row per movie (keep first row)
movie_akas_2 = movie_akas_2.drop_duplicates(subset=['tconst'], keep='first')
movie_akas_2

,tconst,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,\N,\N,\N,0
14,tt0000002,7,The Clown and His Dogs,US,\N,\N,literal English title,0
40,tt0000005,6,Blacksmithing,US,\N,\N,informal alternative title,0
46,tt0000006,3,Chinese Opium Den,US,\N,\N,\N,0
50,tt0000007,1,Corbett and Courtney Before the Kinetograph,US,\N,\N,\N,0
...,...,...,...,...,...,...,...,...
17173152,tt5678950,1,Not Alone: The Life Above,US,\N,\N,\N,0
17173265,tt5678986,1,Secrets and Sins,US,\N,\N,\N,0
17173266,tt5678990,1,A Child's Smile,US,\N,\N,\N,0
17173267,tt5678994,1,Watch What You Ask For,US,\N,\N,\N,0


## Merge Datasets

In [ ]:
# Merge four datasets into one dataframe
movie_data = pd.merge(movie_akas_2, movie_basics, on=["tconst", "tconst"])
movie_data = pd.merge(movie_data, movie_crew, on=["tconst", "tconst"])
movie_data = pd.merge(movie_data, movie_ratings, on=["tconst", "tconst"])

movie_data

,tconst,ordering,title,region,language,types,attributes,isOriginalTitle,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,directors,writers,averageRating,numVotes
0,tt0000001,6,Carmencita,US,\N,\N,\N,0,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",nm0005690,\N,5.6,1648
1,tt0000002,7,The Clown and His Dogs,US,\N,\N,literal English title,0,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",nm0721526,\N,6.1,198
2,tt0000005,6,Blacksmithing,US,\N,\N,informal alternative title,0,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",nm0005690,\N,6.2,2139
3,tt0000006,3,Chinese Opium Den,US,\N,\N,\N,0,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short,nm0005690,\N,5.3,115
4,tt0000007,1,Corbett and Courtney Before the Kinetograph,US,\N,\N,\N,0,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport","nm0374658,nm0005690",\N,5.5,656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327070,tt5678558,1,Sign,US,\N,imdbDisplay,\N,0,short,Sign,Znak,0,2017,\N,25,"Comedy,Drama,Romance",nm7247526,"nm9325525,nm7247526",6.6,14
327071,tt5678628,1,Egotastic FunTime!,US,\N,\N,\N,0,tvSeries,Egotastic FunTime!,Egotastic FunTime!,0,2016,\N,\N,Comedy,nm3412536,nm3412536,8.1,14
327072,tt5678678,1,I'll Be Alone for Christmas,US,\N,\N,\N,0,short,I'll Be Alone for Christmas,I'll Be Alone for Christmas,0,2015,\N,7,"Horror,Short",nm2686595,nm5037245,4.6,9
327073,tt5678732,3,Breaking Point,US,\N,\N,\N,0,movie,Breaking Point,Breaking Point,0,2017,\N,82,"Drama,Thriller",nm6126841,nm6126841,2.7,121


## Clean Merged Data File

In [ ]:
# Understand Numerical Features
movie_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 327075 entries, 0 to 327074
Data columns (total 20 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tconst           327075 non-null  object 
 1   ordering         327075 non-null  int64  
 2   title            327075 non-null  object 
 3   region           327075 non-null  object 
 4   language         327075 non-null  object 
 5   types            327075 non-null  object 
 6   attributes       327075 non-null  object 
 7   isOriginalTitle  327075 non-null  object 
 8   titleType        327075 non-null  object 
 9   primaryTitle     327075 non-null  object 
 10  originalTitle    327075 non-null  object 
 11  isAdult          327075 non-null  int64  
 12  startYear        327075 non-null  object 
 13  endYear          327075 non-null  object 
 14  runtimeMinutes   327075 non-null  object 
 15  genres           327075 non-null  object 
 16  directors        327075 non-null  obje

In [ ]:
#Make copy of df for editing
movie_data_2 = movie_data.copy()

In [ ]:
# Inspect title types
movie_data_2['titleType'].value_counts()

movie           96914
tvEpisode       89782
short           52993
video           32609
tvSeries        23392
tvMovie         14841
tvSpecial        5714
videoGame        5645
tvShort          2659
tvMiniSeries     2526
Name: titleType, dtype: int64

In [ ]:
# Drop all rows with titleTypes that are not movies from the df
movie_data_2 = (movie_data_2.loc[movie_data_2['titleType'] == 'movie'])

In [ ]:
# Drop all adult film rows from the df
movie_data_2 = (movie_data_2.loc[movie_data_2['isAdult'] == 0])

In [ ]:
# Inspect start years
movie_data_2['startYear'].value_counts()

2014    3562
2013    3397
2015    3395
2012    3286
2011    3067
        ... 
1903       2
2021       1
1894       1
1897       1
1904       1
Name: startYear, Length: 126, dtype: int64

In [ ]:
# Replace "\N" with date
movie_data_2 = movie_data_2.replace(r'\\N','1700', regex=True)

# Convert columns to int
movie_data_2.startYear = movie_data_2.startYear.astype(int)

# Filter year column
movie_data_2 = (movie_data_2.loc[movie_data_2['startYear'] > 1950])

# Inspect start years
movie_data_2['startYear'].value_counts()

2014    3562
2013    3397
2015    3395
2012    3286
2011    3067
        ... 
1956     462
1954     456
1960     453
1955     447
2021       1
Name: startYear, Length: 71, dtype: int64

In [ ]:
# Replace 1700 with Nan
movie_data_2 = movie_data_2.replace(r'1700','NaN', regex=True)

In [ ]:
# Drop all rows wih a runtime of NaN
movie_data_2 = (movie_data_2.loc[movie_data_2['runtimeMinutes'] != "NaN"])

# Convert column to int
movie_data_2.runtimeMinutes = movie_data_2.runtimeMinutes.astype(int)

In [ ]:
# Drop all rows wih a director of NaN
movie_data_2 = (movie_data_2.loc[movie_data_2['directors'] != "NaN"])

In [ ]:
# Save tconst column to list to be used for filtering on the raw data files
filter_list = movie_data_2['tconst'].tolist()

## Filter Individual Tables & Export

In [ ]:
# Filter individual data files
movie_akas_clean = movie_akas_2[movie_akas_2['tconst'].isin(filter_list)]
movie_basics_clean = movie_basics[movie_basics['tconst'].isin(filter_list)]
movie_ratings_clean = movie_ratings[movie_ratings['tconst'].isin(filter_list)]
movie_crew_clean = movie_crew[movie_crew['tconst'].isin(filter_list)]

In [ ]:
# Drop unneeded columns from individual data files
movie_akas_clean = movie_akas_clean.drop(columns= 'ordering')
movie_basics_clean = movie_basics_clean.drop(columns= ['isAdult', "endYear"])

In [ ]:
# Save filltered dfs to csv
movie_akas_clean.to_csv('Csv_Output/movie_akas_us.csv', index = True)
movie_basics_clean.to_csv('Csv_Output/movie_basics.csv', index = True)
movie_ratings_clean.to_csv('Csv_Output/movie_ratings.csv', index = True)
movie_crew_clean.to_csv('Csv_Output/movie_crew.csv', index = True)

#### The CSV files will be loaded into the DB and merged. The single merged DB Table will be  connected to for all furture steps

# 1. Import SQL Table

In [ ]:
# Establish Database connection and read in dataset from DB
# pd.read_sql_table('table_name', 'postgres:///db_name')

In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-2.4.6'
spark_version = 'spark-2.'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark


# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-2.4.7-bin-hadoop2.7"


# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,341 kB]
Hit:12 http://ppa.launchpad.net

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://team5-finalproject-resources.s3-us-west-1.amazonaws.com/movie_akas_us.csv"
spark.sparkContext.addFile(url)
movie_akas_us = spark.read.csv(SparkFiles.get("movie_akas_us.csv"), sep=" ", header=True, inferSchema=True)

# Show DataFrame
movie_akas.show(10)

In [ ]:
# Check length of the Data
movie_akas_us.count()

In [ ]:
# Read in title_basics data from S3 Buckets
from pyspark import SparkFiles
url="https://team5-finalproject-resources.s3-us-west-1.amazonaws.com/movie_basics.csv"
spark.sparkContext.addFile(url)
movie_basics = spark.read.csv(SparkFiles.get("movie_basics.csv"), sep=" ", header=True, inferSchema=True)

# Show DataFrame
movie_basics.show(10)

In [ ]:
# Check length of the Data
movie_basics.count()

In [ ]:
# Read in title_akas data from S3 Buckets
from pyspark import SparkFiles
url="https://team5-finalproject-resources.s3-us-west-1.amazonaws.com/movie_ratings.csv"
spark.sparkContext.addFile(url)
movie_ratings = spark.read.csv(SparkFiles.get("movie_ratings.csv"), sep=" ", header=True, inferSchema=True)

# Show DataFrame
movie_ratings.show(10)

In [ ]:
# Count DataFrame
movie_ratings.count()

In [ ]:
# Read in title_crew data from S3 Buckets
from pyspark import SparkFiles
url="https://team5-finalproject-resources.s3-us-west-1.amazonaws.com/movie_crew.csv"
spark.sparkContext.addFile(url)
movie_crew = spark.read.csv(SparkFiles.get("moview_crew.csv"), sep=" ", header=True, inferSchema=True)

# Show DataFrame
movie_crew.show(10)

In [ ]:
# Count DataFrame
movie_crew.count()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Configure settings for RDS 
mode = "append"
jdbc_url="jdbc:postgresql://movies-dataviz.cv7p5opuktsx.us-west-1.rds.amazonaws.com:5432/movie_analysis_db"
config = {"user":"postgres",
          "password":"20202020",
          "driver":"org.postgresql.Driver"}

In [ ]:
movie_akas_us.write.jdbc(url=jdbc_url, table='movie_akas_us', mode=mode, properties=config)

In [ ]:
movie_basics.write.jdbc(url=jdbc_url, table='movie_basics', mode=mode, properties=config)

In [ ]:
movie_ratings.write.jdbc(url=jdbc_url, table='movie_ratings', mode=mode, properties=config)

In [ ]:
movie_crew.write.jdbc(url=jdbc_url, table='movie_crew', mode=mode, properties=config)

# 2. Clean Data File

In [ ]:
# Drop unneeded columns
movie_data_2 = movie_data_2.drop(columns=['title', 'region', 'types', 'tconst', 'titleType', 'originalTitle',
                                          'writers', 'language', 'attributes', 'isOriginalTitle'])
movie_data_2

,ordering,primaryTitle,isAdult,startYear,endYear,runtimeMinutes,genres,directors,averageRating,numVotes
11757,5,Made in Germany - Die dramatische Geschichte d...,0,1957,NaN,101,"Biography,Drama",nm0772191,6.5,12
14126,33,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance",nm0003506,6.4,78045
14424,1,Elephant Fury,0,1953,NaN,100,"Drama,War",nm0682176,4.2,6
14756,12,Mystery of the Black Jungle,0,1954,NaN,80,"Action,Adventure,Mystery","nm0614634,nm0130397",5.4,34
14818,12,"Another Time, Another Place",0,1983,NaN,118,"Drama,War",nm0705535,6.5,256
...,...,...,...,...,...,...,...,...,...,...
327055,1,The Long Way Home: Making the Martian,0,2016,NaN,80,Documentary,nm1361273,7.5,13
327056,1,About Paul,0,2016,NaN,63,"Biography,Documentary",nm2648766,6.1,11
327060,1,Regionrat,0,2019,NaN,99,"Comedy,Drama",nm0721494,6.3,63
327069,2,"Jerico, the Infinite Flight of Days",0,2016,NaN,77,Documentary,nm7058489,7.9,123


In [ ]:
# Rename columns
movie_data_2 = movie_data_2.rename(columns = {"primaryTitle":"Title", "startYear":"Year",
                                              "runtimeMinutes":"Runtime(Min)", "genres":"Genres",
                                              "directors":"Directors", "averageRating":"AverageRating",
                                              "numVotes":"NumVotes"})
movie_data_2

,ordering,Title,isAdult,Year,endYear,Runtime(Min),Genres,Directors,AverageRating,NumVotes
11757,5,Made in Germany - Die dramatische Geschichte d...,0,1957,NaN,101,"Biography,Drama",nm0772191,6.5,12
14126,33,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance",nm0003506,6.4,78045
14424,1,Elephant Fury,0,1953,NaN,100,"Drama,War",nm0682176,4.2,6
14756,12,Mystery of the Black Jungle,0,1954,NaN,80,"Action,Adventure,Mystery","nm0614634,nm0130397",5.4,34
14818,12,"Another Time, Another Place",0,1983,NaN,118,"Drama,War",nm0705535,6.5,256
...,...,...,...,...,...,...,...,...,...,...
327055,1,The Long Way Home: Making the Martian,0,2016,NaN,80,Documentary,nm1361273,7.5,13
327056,1,About Paul,0,2016,NaN,63,"Biography,Documentary",nm2648766,6.1,11
327060,1,Regionrat,0,2019,NaN,99,"Comedy,Drama",nm0721494,6.3,63
327069,2,"Jerico, the Infinite Flight of Days",0,2016,NaN,77,Documentary,nm7058489,7.9,123


In [ ]:
# Make Movie name the Index
movie_data_2 = movie_data_2.set_index('Title')

# 3. Split / Encode Genres & Director Columns

In [ ]:
# Convert genres column from string into a list
movie_data_2["Genres"] = movie_data_2.Genres.apply(lambda x: x.split(','))
movie_data_2

,ordering,isAdult,Year,endYear,Runtime(Min),Genres,Directors,AverageRating,NumVotes
Title,,,,,,,,,
Made in Germany - Die dramatische Geschichte des Hauses Zeiss,5,0,1957,NaN,101,"[Biography, Drama]",nm0772191,6.5,12
Kate & Leopold,33,0,2001,NaN,118,"[Comedy, Fantasy, Romance]",nm0003506,6.4,78045
Elephant Fury,1,0,1953,NaN,100,"[Drama, War]",nm0682176,4.2,6
Mystery of the Black Jungle,12,0,1954,NaN,80,"[Action, Adventure, Mystery]","nm0614634,nm0130397",5.4,34
"Another Time, Another Place",12,0,1983,NaN,118,"[Drama, War]",nm0705535,6.5,256
...,...,...,...,...,...,...,...,...,...
The Long Way Home: Making the Martian,1,0,2016,NaN,80,[Documentary],nm1361273,7.5,13
About Paul,1,0,2016,NaN,63,"[Biography, Documentary]",nm2648766,6.1,11
Regionrat,1,0,2019,NaN,99,"[Comedy, Drama]",nm0721494,6.3,63


In [ ]:
# Use MultiLabelBinarizer() to break up and encode genre list 
mlb = MultiLabelBinarizer()
movie_data_2 = movie_data_2.join(pd.DataFrame(mlb.fit_transform(movie_data_2.pop('Genres')), columns=mlb.classes_, index=movie_data_2.index))
movie_data_2

,ordering,isAdult,Year,endYear,Runtime(Min),Directors,AverageRating,NumVotes,Action,Adult,...,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
Title,,,,,,,,,,,,,,,,,,,,,
!Women Art Revolution,4,0,2010,NaN,83,nm0380961,6.8,210,0,0,...,0,0,0,0,0,0,0,0,0,0
#1 Serial Killer,3,0,2013,NaN,87,nm0950956,5.6,42,0,0,...,0,0,0,0,0,0,0,0,0,0
#5,1,0,2013,NaN,68,nm5361948,6.6,7,0,0,...,0,0,0,0,0,0,0,0,0,0
#50Fathers,1,0,2015,NaN,132,nm3614760,4.6,5,0,0,...,0,0,0,0,0,0,0,0,0,0
#Alive,9,0,2020,NaN,98,nm4252203,6.2,15799,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Últimos días de la víctima,1,0,1982,NaN,90,nm0034899,7.3,386,0,0,...,0,0,0,0,0,0,0,0,0,0
Über Wasser: Menschen und gelbe Kanister,2,0,2007,NaN,120,nm0561059,7.2,45,0,0,...,0,0,0,0,0,0,0,0,0,0
Über uns das All,1,0,2011,NaN,88,nm1099979,6.7,666,0,0,...,0,0,0,0,0,0,0,0,0,0


### We can test the ML model with both dropping the Directors column and encoding it to see the difference in accuracy

In [ ]:
# Inspect directors
movie_data_2['Directors'].value_counts()

nm0001238    85
nm0676248    70
nm0213983    60
nm0000339    59
nm0128715    55
             ..
nm0766112     1
nm0051894     1
nm0112492     1
nm1965648     1
nm2755382     1
Name: Directors, Length: 40511, dtype: int64

In [ ]:
# Convert Directors column from string into a list
movie_data_2["Directors"] = movie_data_2.Directors.apply(lambda x: x.split(','))

## **Please Note: 

### Encoding the Directors column will create over 43,600 new columns, which takes an extremely long time to complete. This process below is currently commented out.

In [ ]:
# Use MultiLabelBinarizer() to break up and encode Directors list 
#movie_data_2 = movie_data_2.join(pd.DataFrame(mlb.fit_transform(movie_data_2.pop('Directors')), columns=mlb.classes_, index=movie_data_2.index))
#movie_data_2

# **4.Machine Learning**
## Split & Standardize Data

## **Please Note: 

### - If the Directors column is encoded (43,600 columns) it will take an extremely long time to split
### - If the Director column is not encoded, it will need to be dropped as you cannot standardize non-numerical values

In [ ]:
# Drop Directors column
movie_data_2 = movie_data_2.drop(columns= 'Directors')
movie_data_2

,ordering,isAdult,Year,endYear,Runtime(Min),AverageRating,NumVotes,Action,Adult,Adventure,...,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
Title,,,,,,,,,,,,,,,,,,,,,
!Women Art Revolution,4,0,2010,NaN,83,6.8,210,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#1 Serial Killer,3,0,2013,NaN,87,5.6,42,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#5,1,0,2013,NaN,68,6.6,7,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#50Fathers,1,0,2015,NaN,132,4.6,5,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#Alive,9,0,2020,NaN,98,6.2,15799,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Últimos días de la víctima,1,0,1982,NaN,90,7.3,386,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Über Wasser: Menschen und gelbe Kanister,2,0,2007,NaN,120,7.2,45,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Über uns das All,1,0,2011,NaN,88,6.7,666,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 4.1 Linear Regresion & Logistic Regression#

In [ ]:
# Split preprocessed data into features and target arrays
y = movie_data_2["AverageRating"].values
X = movie_data_2.["Runtime(Min)"].values

# 4.1.1 Linear Regression

In [7]:
#import Linear Regression dependencies 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [6]:
# Visulize to see if there are any relationships between tow features -- AverageRating and Runtime(Min)
plt.scatter(movie_data_2.Runtime(Min), movie_data_2.AverageRating)
plt.Xlabel("The Runtime of movies")
plt.ylabel("The Average Rating score of movies")
plt.show()

In [ ]:
#Modified by the reshape() method
X = movies_data_2.Runtime(Min).reshape(-1 ,1)
X.shape
y = movies_data_2.AverageRating

In [ ]:
#Create Linear Regression model and fit the featrues
#Visulized the relationship
Linear_Regression_model = LinearRegression()
Linear_Regression_model.fit(X, y)
y_pred = Linear_Regression_model.predict(X)
plt.scatter(X,y)
plt.plot(X, y_pred, color = 'red')
plt.show()
print(Linear_Regression_model.coef_)
print(Linear_Regression_model.intercept_)

# 4.1.2 Logistic Regression

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.datasets import make_blobs
X, y = make_blobs(centers = 2, random_state = 42)

print(f"Labels: {y[:20]}")
print(f"Data: {X[:20]}")

plt.scatter(X[:, 0], X[:,1], c=y)

In [ ]:
#Split the data into train set and test sets
# Split the preprocessed data into a training and testing dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify = y)

In [ ]:
#Scale the datasets

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [ ]:
#Instantiate a Logistic Regression Model
from sklearn.linear_model import LogisticRegression
classifier = LogsiticRegression(solver="lbfgs", random_state =1)
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
   intercept_scaling=1, 11_ratio=None, max_iter=100,
   multi_class='warn', n_jobs=None, penalty='12',
   random_state=1, solver='lbfgs' tol=0.0001, verbose=0,
   warm_start=False)

In [ ]:
#Train the logistic Regression Model 
classifier.fit(X_train, y_train)

In [ ]:
#Validate the Logistic Regression Model and create the dataset
predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction" : predictions, "Actual": y_test})


In [ ]:
#Visualization 
import numpy as np
new_data = np.array([[-2, 6]])
plt.scatter(X[:, 0], X[:, 1], c=y)
plt.scatter(new_data[0, 0], new_data[0, 1], c="r", marker="o", s=100)
plt.show()

In [ ]:
#Print the accuracy scores 
predictions = classifier.predict(new_data)
print("Classes are either 0 (purple) or 1 (yellow)")
print(f"The new point was classified as: {predictions}")
print(accuracy_score(y_test, y_pred))

# **4.1.2.1. Asses Accuracy, Precision, F-1 scores and confusion matrix**

In [ ]:
#confusion matrix, trade-off between accuracy and prediction

from sklearn.metrics import confusion_matrix, classification_report
matrix = confusion_matrix(y_test, y_pred)
print(matrix)

#report the summary 
report = classification_report(y_test, y_pred)
print(report)

# 4.1.3 SVC Model

In [ ]:
#split the model
from sklearn.model_selection import train_test_split
X_train_svc, X_test_svc, y_train_svc, y_test_svc = train_test_split(X,
   y,  random_state=1, stratify=y)
X_train_svc.shape

In [ ]:
#Create the model 
from sklearn.svm import SVC
model = SVC(kernel='linear')

In [ ]:
#Train the model with fit()
model.fit(X_train_svc, y_train_svc)

In [ ]:
#create predictions with the model 
y_pred_svc = model.predict(X_test_svc)
results = pd.DataFrame({
   "Prediction": y_pred,
   "Actual": y_test_svc
}).reset_index(drop=True)
results.head()

In [ ]:
#assess the accuracy_score of the svc model
from sklearn.metrics import accuracy_score
accuracy_score = accuracy_score(y_test_svc, y_pred_svc)
print(accuracy_score)


In [ ]:
#generate a confusion_matrix and print the classfication
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test_svc, y_pred_svc)

from sklearn.metrics import classification_report
print(classification_report(y_test_svc, y_pred_svc))